In [1]:
from sqlalchemy import create_engine
import pandas as pd

In [2]:
engine = create_engine("sqlite:///../data/cms.db", echo=True)
conn = engine.connect()

2021-02-04 20:32:41,105 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2021-02-04 20:32:41,112 INFO sqlalchemy.engine.base.Engine ()
2021-02-04 20:32:41,117 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2021-02-04 20:32:41,119 INFO sqlalchemy.engine.base.Engine ()


In [3]:
conn.execute("""DROP TABLE CMS""")
# Create the table - note that if we use automap_base, the table must have primary key!
conn.execute("""
CREATE TABLE IF NOT EXISTS cms (
    cms_id INTEGER PRIMARY KEY AUTOINCREMENT ,
    facility_id TEXT NOT NULL ,
    facility_name TEXT NOT NULL ,
    address TEXT NOT NULL ,
    city TEXT NOT NULL ,
    state TEXT NOT NULL ,
    state_name TEXT NOT NULL,
    zip_code TEXT NOT NULL ,
    county_name TEXT NOT NULL ,
    measure_id TEXT  NOT NULL ,
    measure_name TEXT  NOT NULL ,
    denominator INTEGER  NOT NULL ,
    score REAL  NOT NULL ,
    lower_estimate REAL  NOT NULL ,
    higher_estimate REAL  NOT NULL ,
    start_date TEXT NOT NULL ,
    end_date TEXT NOT NULL ,
    race_category TEXT NOT NULL,
    household_median_income INTEGER NOT NULL ,
    family_median_income INTEGER NOT NULL ,
    total_population INTEGER NOT NULL ,
    percent_poverty REAL  NOT NULL ,
    percent_veteran REAL  NOT NULL ,
    percent_married REAL  NOT NULL ,
    percent_bachelor REAL  NOT NULL ,
    percent_white REAL  NOT NULL ,
    percent_black REAL  NOT NULL ,
    percent_american_indian REAL  NOT NULL ,
    percent_asian REAL  NOT NULL ,
    percent_hawaiian REAL  NOT NULL ,
    percent_some_other REAL  NOT NULL ,
    percent_two_or_more REAL  NOT NULL ,
    state_code TEXT  NOT NULL ,
    county_code TEXT  NOT NULL 
)
""")

2021-02-04 20:32:41,152 INFO sqlalchemy.engine.base.Engine DROP TABLE CMS
2021-02-04 20:32:41,156 INFO sqlalchemy.engine.base.Engine ()
2021-02-04 20:32:41,461 INFO sqlalchemy.engine.base.Engine COMMIT
2021-02-04 20:32:41,466 INFO sqlalchemy.engine.base.Engine 
CREATE TABLE IF NOT EXISTS cms (
    cms_id INTEGER PRIMARY KEY AUTOINCREMENT ,
    facility_id TEXT NOT NULL ,
    facility_name TEXT NOT NULL ,
    address TEXT NOT NULL ,
    city TEXT NOT NULL ,
    state TEXT NOT NULL ,
    state_name TEXT NOT NULL,
    zip_code TEXT NOT NULL ,
    county_name TEXT NOT NULL ,
    measure_id TEXT  NOT NULL ,
    measure_name TEXT  NOT NULL ,
    denominator INTEGER  NOT NULL ,
    score REAL  NOT NULL ,
    lower_estimate REAL  NOT NULL ,
    higher_estimate REAL  NOT NULL ,
    start_date TEXT NOT NULL ,
    end_date TEXT NOT NULL ,
    race_category TEXT NOT NULL,
    household_median_income INTEGER NOT NULL ,
    family_median_income INTEGER NOT NULL ,
    total_population INTEGER NOT NUL

In [4]:
# Read in target dataframes
census_df = pd.read_csv("../ETL/ETL-Results/census_data.csv", dtype=str)
cms_df = pd.read_csv("../ETL/ETL-Results/four_mort_measures.csv", dtype=str)

In [5]:
# Remove Some Items due to county issues
cms_df = cms_df[cms_df["State"] != "PR"]
cms_df = cms_df[cms_df["State"] != "GU"]
cms_df = cms_df[cms_df["State"] != "MP"]
cms_df = cms_df[cms_df["State"] != "VI"]

In [6]:
len(cms_df)

13478

In [7]:
# Remove Some Items due to county issues
cms_df = cms_df[cms_df["County Name"] != "OBRIEN"]
cms_df = cms_df[cms_df["County Name"] != "JEFFRSON DAVIS"]

cms_df = cms_df[~((cms_df["County Name"] == "LASALLE")
                & (cms_df["State"] == "LA"))]
cms_df = cms_df[~((cms_df["County Name"] == "ST. MARYS")
                & (cms_df["State"] == "MD"))]
cms_df = cms_df[~((cms_df["County Name"] == "MCLEAN")
                & (cms_df["State"] == "ND"))]

In [8]:
len(cms_df)

13461

In [9]:
def fix_county_name(string):

    if string == "DEKALB":
        return "DE KALB"
    if string == "DUPAGE":
        return "DU PAGE"
    elif string == "DISTRICT OF COLUMBIA":
        return "THE DISTRICT"
    elif string == "ANCHORAGE MUNICIPALITY":
        return "ANCHORAGE"
    elif string == "PRINCE GEORGE\'S":
        return "PRINCE GEORGES"
    elif string == "MCDOWELL":
        return "MC DOWELL"
    elif string == "MCHENRY":
        return "MC HENRY"
    elif string == "LASALLE":
        return "LA SALLE"
    elif string == "MCKEAN":
        return "MC KEAN"
    elif string == "MCDUFFIE":
        return "MC DUFFIE"
    elif string == "EAST BATON ROUGE":
        return "E. BATON ROUGE"
    elif string == "LAPORTE":
        return "LA PORTE"
    elif string == "DEWITT":
        return "DE WITT"
    elif string == "JUNEAU CITY AND BOROUGH":
        return "JUNEAU"
    elif string == "KENAI PENINSULA BOROUGH":
        return "KENAI PENINSULA"
    elif string == "MATANUSKA-SUSITNA BOROUGH":
        return "MATANUSKA-SUSITNA"
    elif string == "DESOTO":
        return "DE SOTO"
    elif string == "DOÑA ANA":
        return "DONA ANA"
    elif string == "LA PAZ":
        return "LAPAZ"
    elif string == "MCLEAN":
        return "MC LEAN"
    elif string == "MCCRACKEN":
        return "MC CRACKEN"
    elif string == "MCDONOUGH":
        return "MC DONOUGH"
    elif string == "KETCHIKAN GATEWAY BOROUGH":
        return "KETCHIKAN GATEWAY"
    elif string == "SCOTTS BLUFF":
        return "SCOTT BLUFF"
    elif string == "NORTHUMBERLAND":
        return "NORTHUMBERLND"
    elif string == "MCLENNAN":
        return "MC LENNAN"
    elif string == "MCMINN":
        return "MC MINN"
    elif string == "FAIRBANKS NORTH STAR BOROUGH":
        return "FAIRBANKS NORTH STAR"
    elif string == "MCLEOD":
        return "MC LEOD"
    elif string == "YELLOW MEDICINE":
        return "YELLOW MEDCINE"
    elif string == "SITKA CITY AND BOROUGH":
        return "SITKA BOROUGH"
    else:
        return string

In [10]:
# Save formatted county names
census_df['County Name'] = census_df['County Name'].apply(
    lambda x: fix_county_name(x))

In [11]:
census_df.columns

Index(['County Name', 'State', 'State Abbr', 'Household Median Income',
       'Family's Median Income', 'Total Population', 'Percent Poverty',
       'Percent Veteran', 'Percent Married', 'Percent Bachelor',
       'Percent One Race White', 'Percent One Race Black+',
       'Percent One Race American Indian+', 'Percent One Race Asian',
       'Percent One Race Hawaiian+', 'Percent One Race Some Other',
       'Percent Two Race Or More', 'State Code', 'County Code'],
      dtype='object')

In [12]:
cms_df.columns

Index(['Facility ID', 'Facility Name', 'Address', 'City', 'State', 'ZIP Code',
       'County Name', 'Measure ID', 'Measure Name', 'Denominator', 'Score',
       'Lower Estimate', 'Higher Estimate', 'Start Date', 'End Date'],
      dtype='object')

In [13]:
cms_merged_df = pd.merge(
    cms_df, census_df, 
    how="inner", left_on=["State", "County Name"], right_on=["State Abbr", "County Name"]
)

In [14]:
cms_merged_df.columns

Index(['Facility ID', 'Facility Name', 'Address', 'City', 'State_x',
       'ZIP Code', 'County Name', 'Measure ID', 'Measure Name', 'Denominator',
       'Score', 'Lower Estimate', 'Higher Estimate', 'Start Date', 'End Date',
       'State_y', 'State Abbr', 'Household Median Income',
       'Family's Median Income', 'Total Population', 'Percent Poverty',
       'Percent Veteran', 'Percent Married', 'Percent Bachelor',
       'Percent One Race White', 'Percent One Race Black+',
       'Percent One Race American Indian+', 'Percent One Race Asian',
       'Percent One Race Hawaiian+', 'Percent One Race Some Other',
       'Percent Two Race Or More', 'State Code', 'County Code'],
      dtype='object')

In [15]:
len(cms_merged_df)

13461

In [16]:
cms_merged_df = cms_merged_df.astype({'Percent One Race Black+':'float', 'Percent One Race White': 'float'})

In [17]:
cms_merged_df.dtypes

Facility ID                           object
Facility Name                         object
Address                               object
City                                  object
State_x                               object
ZIP Code                              object
County Name                           object
Measure ID                            object
Measure Name                          object
Denominator                           object
Score                                 object
Lower Estimate                        object
Higher Estimate                       object
Start Date                            object
End Date                              object
State_y                               object
State Abbr                            object
Household Median Income               object
Family's Median Income                object
Total Population                      object
Percent Poverty                       object
Percent Veteran                       object
Percent Ma

In [18]:
black_mean_df = cms_merged_df.groupby(['County Name', 'State Abbr']).agg({'Percent One Race Black+':'mean'})
black_mean_df.head()

,,Percent One Race Black+
County Name,State Abbr,
ABBEVILLE,SC,27.6
ACADIA,LA,17.4
ACCOMACK,VA,28.6
ADA,ID,1.4
ADAIR,KY,3.1


In [19]:
white_mean_df = cms_merged_df.groupby(['County Name', 'State Abbr']).agg({'Percent One Race White':'mean'})
white_mean_df.head()

,,Percent One Race White
County Name,State Abbr,
ABBEVILLE,SC,69.9
ACADIA,LA,79.3
ACCOMACK,VA,68.2
ADA,ID,90.5
ADAIR,KY,94.9


In [20]:
black_mean = black_mean_df['Percent One Race Black+'].mean()
white_mean = white_mean_df['Percent One Race White'].mean()
white_mean

82.93281527531084

In [21]:
black_df = cms_merged_df[cms_merged_df['Percent One Race Black+'] > black_mean]
len(black_df)

5352

In [22]:
white_df = cms_merged_df[cms_merged_df['Percent One Race White'] > white_mean]
len(white_df)

5969

In [23]:
black_df = black_df[~((black_df['Percent One Race White'] > white_mean) & (black_df['Percent One Race Black+'] > black_mean))]
len(black_df)

5159

In [24]:
white_df = white_df[~((white_df['Percent One Race White'] > white_mean) & (white_df['Percent One Race Black+'] > black_mean))]
len(white_df)

5776

In [25]:
black_df = black_df.assign(race_category="B")
black_df.head()

,Facility ID,Facility Name,Address,City,State_x,ZIP Code,County Name,Measure ID,Measure Name,Denominator,...,Percent One Race White,Percent One Race Black+,Percent One Race American Indian+,Percent One Race Asian,Percent One Race Hawaiian+,Percent One Race Some Other,Percent Two Race Or More,State Code,County Code,race_category
304,230089,BEAUMONT HOSPITAL - GROSSE POINTE,468 CADIEUX RD,GROSSE POINTE,MI,48230,WAYNE,MORT_30_COPD,Death rate for COPD patients,496,...,53.1,38.7,0.3,3.4,0.0,1.9,2.5,26,163,B
305,23037F,DETROIT (JOHN D. DINGELL) VA MEDICAL CENTER,4646 JOHN R.,DETROIT,MI,48201,WAYNE,MORT_30_COPD,Death rate for COPD patients,239,...,53.1,38.7,0.3,3.4,0.0,1.9,2.5,26,163,B
306,230104,HARPER UNIVERSITY HOSPITAL,3990 JOHN R STREET,DETROIT,MI,48201,WAYNE,MORT_30_COPD,Death rate for COPD patients,293,...,53.1,38.7,0.3,3.4,0.0,1.9,2.5,26,163,B
307,230270,BEAUMONT HOSPITAL - TAYLOR,10000 TELEGRAPH ROAD,TAYLOR,MI,48180,WAYNE,MORT_30_COPD,Death rate for COPD patients,245,...,53.1,38.7,0.3,3.4,0.0,1.9,2.5,26,163,B
308,230142,BEAUMONT HOSPITAL - WAYNE,33155 ANNAPOLIS AVE,WAYNE,MI,48184,WAYNE,MORT_30_COPD,Death rate for COPD patients,311,...,53.1,38.7,0.3,3.4,0.0,1.9,2.5,26,163,B


In [26]:
white_df = white_df.assign(race_category="W")
white_df.head()

,Facility ID,Facility Name,Address,City,State_x,ZIP Code,County Name,Measure ID,Measure Name,Denominator,...,Percent One Race White,Percent One Race Black+,Percent One Race American Indian+,Percent One Race Asian,Percent One Race Hawaiian+,Percent One Race Some Other,Percent Two Race Or More,State Code,County Code,race_category
298,140034,SSM HEALTH ST MARY'S HOSPITAL -CENTRALIA,400 NORTH PLEASANT AVENUE,CENTRALIA,IL,62801,MARION,MORT_30_COPD,Death rate for COPD patients,531,...,92.5,3.7,0.2,0.6,0.0,0.5,2.5,17,121,W
299,141345,SALEM TOWNSHIP HOSPITAL,1201 RICKER DRIVE,SALEM,IL,62881,MARION,MORT_30_COPD,Death rate for COPD patients,47,...,92.5,3.7,0.2,0.6,0.0,0.5,2.5,17,121,W
300,140034,SSM HEALTH ST MARY'S HOSPITAL -CENTRALIA,400 NORTH PLEASANT AVENUE,CENTRALIA,IL,62801,MARION,MORT_30_HF,Death rate for heart failure patients,304,...,92.5,3.7,0.2,0.6,0.0,0.5,2.5,17,121,W
301,140034,SSM HEALTH ST MARY'S HOSPITAL -CENTRALIA,400 NORTH PLEASANT AVENUE,CENTRALIA,IL,62801,MARION,MORT_30_AMI,Death rate for heart attack patients,51,...,92.5,3.7,0.2,0.6,0.0,0.5,2.5,17,121,W
302,141345,SALEM TOWNSHIP HOSPITAL,1201 RICKER DRIVE,SALEM,IL,62881,MARION,MORT_30_PN,Death rate for pneumonia patients,70,...,92.5,3.7,0.2,0.6,0.0,0.5,2.5,17,121,W


In [27]:
white_black_df = pd.concat([white_df, black_df])
len(white_black_df)

10935

In [28]:
white_black_df = white_black_df.rename(
        columns={
            "Facility ID": "facility_id",
            "Facility Name": "facility_name",
            "Address": "address",
            "City": "city",
            "State_x": "state",
            "State_y": "state_name",
            "ZIP Code": "zip_code",
            "County Name": "county_name",
            "Measure ID": "measure_id",
            "Measure Name": "measure_name",
            "Denominator": "denominator",
            "Score": "score",
            "Lower Estimate": "lower_estimate",
            "Higher Estimate": "higher_estimate",
            "Start Date": "start_date",
            "End Date": "end_date",
            "Household Median Income": "household_median_income",
            "Family's Median Income": "family_median_income",
            "Total Population": "total_population",
            "Percent Poverty": "percent_poverty",
            "Percent Veteran": "percent_veteran",
            "Percent Married": "percent_married",
            "Percent Bachelor": "percent_bachelor",
            "Percent One Race White": "percent_white",
            "Percent One Race Black+": "percent_black",
            "Percent One Race American Indian+": "percent_american_indian",
            "Percent One Race Asian": "percent_asian",
            "Percent One Race Hawaiian+": "percent_hawaiian",
            "Percent One Race Some Other": "percent_some_other",
            "Percent Two Race Or More": "percent_two_or_more",
            "State Code": "state_code",
            "County Code": "county_code"
        }
)

In [29]:
white_black_df.columns

Index(['facility_id', 'facility_name', 'address', 'city', 'state', 'zip_code',
       'county_name', 'measure_id', 'measure_name', 'denominator', 'score',
       'lower_estimate', 'higher_estimate', 'start_date', 'end_date',
       'state_name', 'State Abbr', 'household_median_income',
       'family_median_income', 'total_population', 'percent_poverty',
       'percent_veteran', 'percent_married', 'percent_bachelor',
       'percent_white', 'percent_black', 'percent_american_indian',
       'percent_asian', 'percent_hawaiian', 'percent_some_other',
       'percent_two_or_more', 'state_code', 'county_code', 'race_category'],
      dtype='object')

In [30]:
white_black_df = white_black_df[[
    "facility_id",
    "facility_name",
    "address",
    "city",
    "state",
    "state_name",
    "zip_code",
    "county_name",
    "measure_id",
    "measure_name",
    "denominator",
    "score",
    "lower_estimate",
    "higher_estimate",
    "start_date",
    "end_date",
    "race_category",
    "household_median_income",
    "family_median_income",
    "total_population",
    "percent_poverty",
    "percent_veteran",
    "percent_married",
    "percent_bachelor",
    "percent_white",
    "percent_black",
    "percent_american_indian",
    "percent_asian",
    "percent_hawaiian",
    "percent_some_other",
    "percent_two_or_more",
    "state_code",
    "county_code"        
]]

In [31]:
white_black_df.columns

Index(['facility_id', 'facility_name', 'address', 'city', 'state',
       'state_name', 'zip_code', 'county_name', 'measure_id', 'measure_name',
       'denominator', 'score', 'lower_estimate', 'higher_estimate',
       'start_date', 'end_date', 'race_category', 'household_median_income',
       'family_median_income', 'total_population', 'percent_poverty',
       'percent_veteran', 'percent_married', 'percent_bachelor',
       'percent_white', 'percent_black', 'percent_american_indian',
       'percent_asian', 'percent_hawaiian', 'percent_some_other',
       'percent_two_or_more', 'state_code', 'county_code'],
      dtype='object')

In [32]:
white_black_df.to_sql("cms", conn, if_exists="append", index=False)

2021-02-04 20:32:44,260 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_info("cms")
2021-02-04 20:32:44,263 INFO sqlalchemy.engine.base.Engine ()
2021-02-04 20:32:44,349 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2021-02-04 20:32:45,735 INFO sqlalchemy.engine.base.Engine INSERT INTO cms (facility_id, facility_name, address, city, state, state_name, zip_code, county_name, measure_id, measure_name, denominator, score, lower_estimate, higher_estimate, start_date, end_date, race_category, household_median_income, family_median_income, total_population, percent_poverty, percent_veteran, percent_married, percent_bachelor, percent_white, percent_black, percent_american_indian, percent_asian, percent_hawaiian, percent_some_other, percent_two_or_more, state_code, county_code) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
2021-02-04 20:32:45,737 INFO sqlalchemy.engine.base.Engine (('140034', "SSM HEALTH ST MARY'S HOSPITAL